In [ ]:
import tensorflow as tf
import numpy as np
from cumt_eye import Cumt_itraker
import matplotlib.pyplot as plt
sess=tf.InteractiveSession()
cumt_data=np.load('cumt_data.pkl')

In [ ]:
X=tf.placeholder(shape=[None,32,128,3],dtype=tf.float32)
Y=tf.placeholder(shape=[None,10],dtype=tf.float32)
DROPOUT=tf.placeholder(dtype=tf.float32)
LR=tf.placeholder(dtype=tf.float32)
BN_FLAG=tf.placeholder(dtype=tf.bool)

In [ ]:
with tf.name_scope('model'):
    model=Cumt_itraker(image=X,bn_flag=BN_FLAG,dropout_rate=DROPOUT)
    y_score=model.score

In [ ]:
with tf.name_scope('LOSS'):
    LOSS=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_score,labels=Y))
    tf.summary.scalar('loss',LOSS)
with tf.name_scope('TRAIN'):
    TRAIN=tf.train.AdamOptimizer(LR).minimize(LOSS)
with tf.name_scope('ACCURACY'):
    acc_count=tf.equal(tf.arg_max(y_score,1),tf.arg_max(Y,1))
    ACCURACY=tf.reduce_mean(tf.cast(acc_count,tf.float32))
    tf.summary.scalar('acc',ACCURACY)

In [ ]:
writer_tr=tf.summary.FileWriter('./mylog/test')
writer_te=tf.summary.FileWriter('./mylog/val')
merge=tf.summary.merge_all()
ex_ops=tf.get_collection(tf.GraphKeys.UPDATE_OPS)



In [ ]:
#初始化变量，需要重新训练才运行
init=tf.global_variables_initializer()
sess.run(init)

In [ ]:
#从保存的检查点钟复原变量
saver=tf.train.Saver()
saver.restore(sess,'./model_save/model_71.ckpt-64600')

In [ ]:
test_range=np.random.choice(range(cumt_data['data'].shape[0]),int(cumt_data['data'].shape[0]*0.2),replace=False)
train_range=np.asarray(list(set(range(cumt_data['data'].shape[0]))-set(test_range)))

In [ ]:
d=np.load('eye_val.pkl')
val={}
val['images']=d['images'].astype('float32')-[103.939, 116.779, 123.68]
val['labels']=d['labels']
del d

In [ ]:
saver=tf.train.Saver()

In [ ]:
lr_=1e-4
for i in range(1000*6*12):
    mask=np.random.choice(train_range,256,replace=False)
    x_,y_=cumt_data['data'][mask],cumt_data['labels'][mask]
    feed_dict={X:x_,Y:y_,DROPOUT:0.3,BN_FLAG:True,LR:lr_}
    sess.run([TRAIN,ex_ops],feed_dict=feed_dict)
#     if i%10==0:
#         loss_,acc_,m_=sess.run([LOSS,ACCURACY,merge],feed_dict=feed_dict)
#         writer_tr.add_summary(m_,i)
#         print('epoch:{},loss:{},accuracy:{}'.format(i,loss_,acc_))
    if i%50==0:
        mask=np.random.choice(test_range,256,replace=False)
        x_,y_=cumt_data['data'][mask],cumt_data['labels'][mask]
        feed_dict={X:x_,Y:y_,DROPOUT:1.,BN_FLAG:False,LR:lr_}
        loss_,acc_,m_=sess.run([LOSS,ACCURACY,merge],feed_dict=feed_dict)
        writer_tr.add_summary(m_,i)
        print('epoch:{},loss:{},accuracy:{}'.format(i,loss_,acc_))
    if i%10==0:
        x_,y_=val['images'],val['labels']
        feed_dict={X:x_,Y:y_,DROPOUT:1.,BN_FLAG:False,LR:lr_}
        loss_,acc_,m_=sess.run([LOSS,ACCURACY,merge],feed_dict=feed_dict)
        writer_te.add_summary(m_,i)
        if acc_>0.7:
            m_name='./model_save/model2_'+str(int(acc_*100))+'.ckpt'
            saver.save(sess,m_name,global_step=i)
            lr_=max(1e-5,0.97*lr_)
            print('Save model,epoch:{},validate accuracy:{},learning rate:{}'.format(i,acc_,lr_))
        print('--epoch:{},loss:{},accuracy:{}'.format(i,loss_,acc_))

In [ ]:
#保存模型
saver=tf.train.Saver()
saver.save(sess,'./model_save/model.ckpt',global_step=1000)

In [ ]:
map_lab={ 
        '7':4,'8':3,'9':2,
        '4':7,'5':6,'6':5,
        '1':10,'2':9,'3':8,
'0':1}
         

In [ ]:
prob=tf.nn.softmax(y_score)

In [ ]:
import os
from scipy.misc import imread
from scipy.misc import imresize

In [ ]:
def getimg(addr):
    VGG_MEAN = [103.939, 116.779, 123.68]
    a=imread(addr)
    plt.imshow(a)
    plt.show()
    b=imresize(a-VGG_MEAN,(32,128,3)).reshape((1,32,128,3))
    return b

In [ ]:
img_dir=r'D:\Proj_DL\Code\Proj_EyeTraker\eye_val\441_5.jpg'
b=getimg(img_dir)

In [ ]:
p_=sess.run(prob,feed_dict={X:b,DROPOUT:1.,BN_FLAG:False,LR:1e-3})

In [ ]:
list(p_)

In [ ]:
np.argmax(p_)+1